In [ ]:
import numpy as np
from gaussians import Gaussians
import json
import cv2
import pickle
import torch
from torch import nn
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
from modules.bilinear_interpolation import BilinearInterpolation

In [ ]:
with open('train.json') as f:
    data = json.load(f)

In [ ]:
heatmap_list = []

for i in range(16):
    img_name = data[i]['image']

    # Change the folder here
    img = cv2.imread('mpii_16/' + img_name)
    kpt = np.asarray(data[i]['joints'], dtype=np.int32)
    if img.shape[0] != 368 or img.shape[1] != 368:
        kpt[:, 0] = kpt[:, 0] * (368 / img.shape[1])
        kpt[:, 1] = kpt[:, 1] * (368 / img.shape[0])
        img = cv2.resize(img, (368, 368))
        img = np.array(img)

    gaussian = Gaussians()
    g_list = []
    g_list.append(kpt)
    device = "cuda"

    # Pickled model input
    filename = 'epoch.pkl'
    infile = open(filename,'rb')
    model = pickle.load(infile).to(device)
    img_cuda = torch.Tensor(img).unsqueeze(0).permute(0, 3, 1, 2).to(device)
    output = model(img_cuda)
    softmax = nn.Softmax(dim=2)

    # View the 
    input_view = output.view(output.shape[0], output.shape[1], -1)
    output_view = softmax(input_view)
    output = output_view.view(output.shape)
    infile.close()
    bilin = BilinearInterpolation(output_size = (368, 368))
    output = bilin(output)

    outmap = output.cpu().detach().numpy()

    heatmapmodel = outmap * 255.0 
    heatmap_list.append(np.transpose(heatmapmodel[0,0]))

In [ ]:
cv2_imshow(heatmap_list[3])